In [ ]:
#Connect to google drive to access the files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#library imports
  #required to run the ML and the preprocessing stages, plus visualization
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import seaborn as sns
import os
from matplotlib.pyplot import figure
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from keras import Sequential
from keras import optimizers
from keras.optimizers import SGD



In [ ]:
df_data = pd.read_csv('/content/drive/MyDrive/ESOF-4969 Degree Project/Machine Learning/Production_Data_final.csv')#read the dataset

In [ ]:
df_data.head()

,Unnamed: 0,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,...,Channel2_318,Channel2_319,Channel2_320,Channel2_321,Channel2_322,Channel2_323,Channel2_324,Channel2_325,Channel2_326,Channel2_327
0,0,37Jdi2cCSRA5jl6zADeLOy,D R O V E MY CAR,1 Trait High,https://p.scdn.co/mp3-preview/ab54f8f789175d42...,1 Trait Danger,comedy rap,120937,0.601,0.827,...,896,-18226,-5812,4875,-7218,8262,14948,-10,8951,-6552
1,1,14jLrcO0NGIxtX56LHEhsv,Reprisal,1 Trait World Tour,https://p.scdn.co/mp3-preview/ffe73947f36e6e15...,1 Trait Danger,comedy rap,100634,0.391,0.575,...,-1289,2746,1653,614,1316,-472,3823,672,-251,2903
2,2,7dcNDXD9SwJ4Txqg2fE2t8,Danger,Back from the Dead,https://p.scdn.co/mp3-preview/1ae38bb85e0313f6...,LUCKY3RD,houston rap,181395,0.802,0.511,...,-1197,-1213,-3706,-1938,-3796,361,-140,1618,-494,-1781
3,3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.569,...,3429,9928,-26812,10141,17013,16055,16658,12518,19679,8613
4,4,1wDY9VWdCoRjLV4gUyYf52,Danger,In Transit (Deluxe),https://p.scdn.co/mp3-preview/61586296e9713d07...,Terri,nigerian pop,157648,0.514,0.582,...,-220,-6064,9413,4410,-177,507,6667,-8546,-20569,2043


In [ ]:
df_data_better = df_data.drop(['Unnamed: 0'],axis=1) #Drops the unnamed column which is added when combining csv

In [ ]:
df_trimmed = df_data_better.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre','Duration'], axis=1)
df_trimmed
#training data

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,...,Channel2_318,Channel2_319,Channel2_320,Channel2_321,Channel2_322,Channel2_323,Channel2_324,Channel2_325,Channel2_326,Channel2_327
0,0.601,0.827,2,-7.056,0.1350,0.01180,0.000000,0.9610,0.2930,128.055,...,896,-18226,-5812,4875,-7218,8262,14948,-10,8951,-6552
1,0.391,0.575,5,-7.520,0.1070,0.38000,0.004660,0.4130,0.3260,143.260,...,-1289,2746,1653,614,1316,-472,3823,672,-251,2903
2,0.802,0.511,6,-9.651,0.2980,0.03120,0.000000,0.0843,0.3320,84.979,...,-1197,-1213,-3706,-1938,-3796,361,-140,1618,-494,-1781
3,0.720,0.569,5,-6.983,0.0607,0.01650,0.000525,0.5400,0.1690,119.989,...,3429,9928,-26812,10141,17013,16055,16658,12518,19679,8613
4,0.514,0.582,11,-5.243,0.3880,0.35500,0.000001,0.1840,0.4530,66.681,...,-220,-6064,9413,4410,-177,507,6667,-8546,-20569,2043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.316,0.110,9,-17.251,0.0367,0.99200,0.539000,0.4030,0.1220,88.791,...,-1421,-745,3733,5421,541,463,-1944,1936,-11179,-4804
71147,0.355,0.362,1,-13.699,0.0340,0.70900,0.623000,0.1220,0.3570,86.946,...,776,-2015,1290,1881,-5497,-811,-4471,120,3245,8489
71148,0.591,0.973,1,-2.835,0.0471,0.00420,0.390000,0.1850,0.0419,125.974,...,-19519,-13232,-8683,-745,-2270,10022,661,-12666,2892,-6850
71149,0.522,0.988,1,-4.646,0.3210,0.00320,0.051300,0.1200,0.0689,141.685,...,7484,3477,6979,-7515,-14778,12483,12314,-1127,7092,-2415


In [ ]:
#df_trimmed.iloc[:,677] #first channel col they go from 24 - 677   so encode 24 - 238, 239 - 478, 479 - 677

In [ ]:
df_autoencode_col1 = df_trimmed.iloc[:,24:238] 
df_autoencode_col1

,Channel1_1,Channel1_2,Channel1_3,Channel1_4,Channel1_5,Channel1_6,Channel1_7,Channel1_8,Channel1_9,Channel1_10,...,Channel1_205,Channel1_206,Channel1_207,Channel1_208,Channel1_209,Channel1_210,Channel1_211,Channel1_212,Channel1_213,Channel1_214
0,-4939,4194,-17352,-11871,7825,-3263,415,14806,11231,-9412,...,-11733,2678,8387,5506,3732,12513,-7228,22984,5390,-1772
1,-2246,11016,3776,-3226,-3583,-2998,302,3774,1259,10551,...,-3657,10186,1503,13885,7763,5730,12272,1130,-1511,-3637
2,-3223,-12447,-9906,15064,14835,-9678,-7472,-21299,-2118,-406,...,3471,1735,3138,5581,6563,-929,-586,-1014,2864,-2069
3,-13728,-17200,-25698,-20324,-15013,-6440,-838,1555,2313,3070,...,-13610,-13448,-20283,-17177,-24091,-18966,2666,14508,15417,14645
4,6464,7954,-1882,14061,29068,-2079,6771,-3701,-8761,4913,...,7837,5888,-18432,5419,22291,-19052,-4096,24170,-3508,-12830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,4662,-511,60,3401,-1365,1575,-2072,-2090,2602,1726,...,555,-308,-1245,-423,-1170,657,704,138,175,-531
71147,-5918,2886,8858,7088,1158,2018,205,3419,-1652,1861,...,-11204,-1157,-1099,-4230,9323,6413,-5064,-600,-4319,-344
71148,5313,-583,-3981,-6023,-2219,4271,2331,3638,496,-5468,...,-8999,3184,20605,-11299,8704,27786,-21904,-7278,5154,8945
71149,-7473,-1399,17234,18489,2763,12567,8296,9652,-624,-5688,...,-3084,-4161,5835,-10751,-12252,-1748,-9073,-3636,-2501,6878


In [ ]:
df_autoencode_col2 = df_trimmed.iloc[:,238:458] 
df_autoencode_col2

,Channel1_215,Channel1_216,Channel1_217,Channel1_218,Channel1_219,Channel1_220,Channel1_221,Channel1_222,Channel1_223,Channel1_224,...,Channel2_98,Channel2_99,Channel2_100,Channel2_101,Channel2_102,Channel2_103,Channel2_104,Channel2_105,Channel2_106,Channel2_107
0,3688,17333,-7104,9718,12091,8191,-7243,18101,-3726,2681,...,9852,-13160,12145,16151,10696,-18477,12583,-6585,6214,2670
1,543,3534,-2267,2661,-7165,-18708,-14092,-17268,-12652,-132,...,-4624,-10116,-13450,-6970,-12398,-17286,-8346,-16259,-69,501
2,-5040,7644,1865,-718,-12854,17857,-21657,8285,-1187,4825,...,-2807,-1657,7950,16212,-11512,3245,7834,-19571,13773,-10901
3,2162,19838,12908,13220,14433,10498,9972,22515,12238,19649,...,4754,-3005,9848,-18386,-16777,-13692,-14870,-9948,-9557,-11671
4,12929,22429,-12469,3131,20567,-4343,5313,-3357,2023,-3168,...,-2163,-19183,9031,4075,-24119,1962,14064,-17832,-1985,6982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,444,1135,-177,-5747,104,1661,-4222,-2963,2467,466,...,164,695,7628,-480,-986,-3705,6255,3693,7504,6166
71147,-235,-12410,-6436,1540,-8401,-248,4600,-2976,-247,-3432,...,647,-6495,3410,8522,933,1293,-555,-1310,-918,-5172
71148,-5676,-167,-3853,-1153,1811,10113,15844,-782,10565,7230,...,-19748,7600,21309,-362,2509,24638,3922,16662,-27042,4234
71149,26,-6392,5310,-2978,2000,500,-7743,-1773,14624,-19716,...,9442,-8481,-15540,-694,1631,7676,-12455,20038,6304,1117


In [ ]:
df_autoencode_col3 = df_trimmed.iloc[:,458:678] 
df_autoencode_col3

,Channel2_108,Channel2_109,Channel2_110,Channel2_111,Channel2_112,Channel2_113,Channel2_114,Channel2_115,Channel2_116,Channel2_117,...,Channel2_318,Channel2_319,Channel2_320,Channel2_321,Channel2_322,Channel2_323,Channel2_324,Channel2_325,Channel2_326,Channel2_327
0,10492,1507,5264,13494,-12382,-142,19221,3157,7951,13971,...,896,-18226,-5812,4875,-7218,8262,14948,-10,8951,-6552
1,4125,8630,6377,11607,4337,4258,17227,-185,7723,5761,...,-1289,2746,1653,614,1316,-472,3823,672,-251,2903
2,13753,-11993,935,13271,-11406,-390,11122,400,926,9413,...,-1197,-1213,-3706,-1938,-3796,361,-140,1618,-494,-1781
3,4301,2385,8580,-10836,-10780,-6050,-21256,-23524,-15624,4939,...,3429,9928,-26812,10141,17013,16055,16658,12518,19679,8613
4,-2307,-16732,7752,3314,-20300,17859,394,1011,-17488,5885,...,-220,-6064,9413,4410,-177,507,6667,-8546,-20569,2043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,-7918,4202,-4206,978,9505,-3282,5599,-6297,7025,1100,...,-1421,-745,3733,5421,541,463,-1944,1936,-11179,-4804
71147,4537,-11203,-4601,2595,-206,1539,-2105,3098,-7465,3930,...,776,-2015,1290,1881,-5497,-811,-4471,120,3245,8489
71148,-12722,-13067,26263,14152,11096,23214,-6168,16421,-3986,14406,...,-19519,-13232,-8683,-745,-2270,10022,661,-12666,2892,-6850
71149,9811,-13090,783,6220,13331,6745,18,4986,2501,9575,...,7484,3477,6979,-7515,-14778,12483,12314,-1127,7092,-2415


NeuralNet stuff

sklearn SOM

In [ ]:
pip install sklearn-som


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn_som.som import SOM

minmaxscaling all dfs


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range= (0,1))
df_trimmed=sc.fit_transform(df_trimmed)
df_trimmed

array([[0.60953347, 0.827     , 0.18181818, ..., 0.49985504, 0.63659113,
        0.40003052],
       [0.39655172, 0.575     , 0.45454545, ..., 0.51026169, 0.49617762,
        0.54430457],
       [0.81338742, 0.511     , 0.54545455, ..., 0.52469673, 0.49246967,
        0.47283131],
       ...,
       [0.59939148, 0.973     , 0.09090909, ..., 0.30673686, 0.54413672,
        0.39548333],
       [0.52941176, 0.988     , 0.09090909, ..., 0.48281071, 0.60822461,
        0.46315709],
       [0.94827586, 0.484     , 0.45454545, ..., 0.52785534, 0.49982452,
        0.49941253]])

In [ ]:

df_autoencode_col1=sc.fit_transform(df_autoencode_col1)
df_autoencode_col1

array([[0.42464332, 0.56400397, 0.23523308, ..., 0.85072099, 0.58225376,
        0.47296864],
       [0.46573587, 0.66810101, 0.5576257 , ..., 0.51725032, 0.47695125,
        0.44451057],
       [0.4508278 , 0.31007858, 0.34885176, ..., 0.48453498, 0.54370947,
        0.46843671],
       ...,
       [0.58107881, 0.49111162, 0.43926146, ..., 0.38895247, 0.57865263,
        0.63649958],
       [0.38597696, 0.47866026, 0.76298161, ..., 0.44452583, 0.46184482,
        0.60495918],
       [0.61886015, 0.55141527, 0.48647288, ..., 0.49977874, 0.56450752,
        0.49864958]])

In [ ]:

df_autoencode_col2=sc.fit_transform(df_autoencode_col2)
df_autoencode_col2

array([[0.5562829 , 0.76449226, 0.39160754, ..., 0.39952697, 0.59482719,
        0.54074922],
       [0.50829328, 0.55393301, 0.46541543, ..., 0.25191119, 0.49895476,
        0.5076524 ],
       [0.42310216, 0.61664759, 0.52846571, ..., 0.20137331, 0.71017014,
        0.33366903],
       ...,
       [0.41339742, 0.49745937, 0.44121462, ..., 0.75425345, 0.08737316,
        0.56461433],
       [0.50040436, 0.40247196, 0.58103304, ..., 0.80576791, 0.5962005 ,
        0.51705196],
       [0.50008392, 0.49918364, 0.39714656, ..., 0.29359884, 0.47452506,
        0.52034791]])

In [ ]:

df_autoencode_col3=sc.fit_transform(df_autoencode_col3)
df_autoencode_col3

array([[0.66010529, 0.52300298, 0.58033112, ..., 0.49985504, 0.63659113,
        0.40003052],
       [0.56295109, 0.63169299, 0.59731441, ..., 0.51026169, 0.49617762,
        0.54430457],
       [0.70986496, 0.31700618, 0.51427481, ..., 0.52469673, 0.49246967,
        0.47283131],
       ...,
       [0.30588235, 0.30061799, 0.90075532, ..., 0.30673686, 0.54413672,
        0.39548333],
       [0.64971389, 0.30026703, 0.51195544, ..., 0.48281071, 0.60822461,
        0.46315709],
       [0.49254597, 0.53904021, 0.48369574, ..., 0.52785534, 0.49982452,
        0.49941253]])

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LeakyReLU

In [ ]:
input1 = Input(shape=df_autoencode_col1.shape[1:]) #gets the size of the number of features in the df i am pulling from
encoder = Dense(128)(input1) 
encoder = LeakyReLU(alpha=0.15)(encoder)#leakyrelu used in hidden layers
encoder = Dense(64)(encoder)
encoder = LeakyReLU(alpha=0.15)(encoder)
# latent space with tanh actiavtion
latent_space1 = Dense(22, activation="tanh")(encoder) #bottleneck layer, used

decoder = Dense(64)(latent_space1)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(128)(decoder)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(units=df_autoencode_col1.shape[1], activation="tanh")(decoder) #never used as pulling from latent space

autoencoder = Model(input1, decoder)
autoencoder.compile(optimizer = "adam", metrics=["mse"], loss="mse")#compiling model with the neccesary optimizer and loss
autoencoder.fit(df_autoencode_col1, df_autoencode_col1, epochs=12, batch_size=25, validation_split=0.2)#fits the autoencoder, 12 epoch, 25 batch size

Epoch 1/12
2277/2277 [==============================] - 25s 7ms/step - loss: 0.0193 - mse: 0.0193 - val_loss: 0.0122 - val_mse: 0.0122
Epoch 2/12
2277/2277 [==============================] - 12s 5ms/step - loss: 0.0172 - mse: 0.0172 - val_loss: 0.0118 - val_mse: 0.0118
Epoch 3/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0169 - mse: 0.0169 - val_loss: 0.0117 - val_mse: 0.0117
Epoch 4/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0168 - mse: 0.0168 - val_loss: 0.0117 - val_mse: 0.0117
Epoch 5/12
2277/2277 [==============================] - 12s 5ms/step - loss: 0.0167 - mse: 0.0167 - val_loss: 0.0117 - val_mse: 0.0117
Epoch 6/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0167 - mse: 0.0167 - val_loss: 0.0116 - val_mse: 0.0116
Epoch 7/12
2277/2277 [==============================] - 12s 5ms/step - loss: 0.0167 - mse: 0.0167 - val_loss: 0.0118 - val_mse: 0.0118
Epoch 8/12
2277/2277 [==============================] -

In [ ]:
encoder1 = Model(input1, latent_space1)

In [ ]:
test_au_features1 = encoder1.predict(df_autoencode_col1)
test_au_features1 #will be used for the final autoencoder to make the 16 attribute encoder

2224/2224 [==============================] - 3s 1ms/step


array([[-0.05643916, -0.06152062, -0.02819233, ...,  0.25680742,
        -0.06328538,  0.21314065],
       [ 0.04491898, -0.14818682, -0.06462201, ...,  0.10478102,
        -0.26085272,  0.09377602],
       [ 0.02229386, -0.11734653, -0.07328019, ...,  0.01642459,
         0.01562475,  0.1934401 ],
       ...,
       [ 0.05773264,  0.27494836, -0.28864425, ...,  0.12209412,
         0.00075674,  0.2893919 ],
       [ 0.04207512,  0.12421777, -0.22763883, ..., -0.10825127,
         0.088888  ,  0.0751533 ],
       [ 0.12209867,  0.15779342, -0.18905002, ..., -0.18775402,
         0.06668454, -0.05110889]], dtype=float32)

encoder #2

In [ ]:
input2 = Input(shape=df_autoencode_col2.shape[1:]) #same as last encoder but differnt dataframe
encoder = Dense(128)(input2)
encoder = LeakyReLU(alpha=0.15)(encoder)
encoder = Dense(64)(encoder)
encoder = LeakyReLU(alpha=0.15)(encoder)

latent_space2 = Dense(22, activation="tanh")(encoder)

decoder = Dense(64)(latent_space2)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(128)(decoder)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(units=df_autoencode_col2.shape[1], activation="tanh")(decoder)

autoencoder = Model(input2, decoder)
autoencoder.compile(optimizer = "adam",metrics=["mse"],loss="mse")
autoencoder.fit(df_autoencode_col2, df_autoencode_col2, epochs=12, batch_size=25, validation_split=0.2)

Epoch 1/12
2277/2277 [==============================] - 14s 5ms/step - loss: 0.0185 - mse: 0.0185 - val_loss: 0.0116 - val_mse: 0.0116
Epoch 2/12
2277/2277 [==============================] - 12s 5ms/step - loss: 0.0165 - mse: 0.0165 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 3/12
2277/2277 [==============================] - 12s 5ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 4/12
2277/2277 [==============================] - 13s 6ms/step - loss: 0.0160 - mse: 0.0160 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 5/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0160 - mse: 0.0160 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 6/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0160 - mse: 0.0160 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 7/12
2277/2277 [==============================] - 12s 5ms/step - loss: 0.0159 - mse: 0.0159 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 8/12
2277/2277 [==============================] -

In [ ]:
encoder2 = Model(input2, latent_space2)

In [ ]:
test_au_features2 = encoder2.predict(df_autoencode_col2) #used in third autoencoder

2224/2224 [==============================] - 4s 2ms/step


In [ ]:
test_au_features2

array([[-0.22561713,  0.16391201, -0.26173645, ..., -0.06500369,
         0.2057669 ,  0.15285553],
       [-0.00797063,  0.24751163, -0.41413668, ..., -0.04060416,
        -0.09857869,  0.18279773],
       [-0.13220352,  0.13492098, -0.15828153, ...,  0.14045997,
         0.00557433, -0.0473071 ],
       ...,
       [-0.17750284, -0.0995979 , -0.28767297, ...,  0.10975104,
         0.19008397,  0.06029651],
       [-0.12658715, -0.24861823, -0.07316612, ..., -0.02438308,
        -0.08587472,  0.04325487],
       [-0.12592955,  0.10966748, -0.150471  , ...,  0.06137919,
         0.08196014,  0.04532818]], dtype=float32)

encoder 3

In [ ]:
input3 = Input(shape=df_autoencode_col3.shape[1:])
encoder = Dense(128)(input3)
encoder = LeakyReLU(alpha=0.15)(encoder)
encoder = Dense(64)(encoder)
encoder = LeakyReLU(alpha=0.15)(encoder)
latent_space3 = Dense(22, activation="tanh")(encoder)

decoder = Dense(64)(latent_space3)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(128)(decoder)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(units=df_autoencode_col3.shape[1], activation="tanh")(decoder)

autoencoder = Model(input3, decoder)
autoencoder.compile(optimizer = "adam",metrics=["mse"],loss="mse")
autoencoder.fit(df_autoencode_col3, df_autoencode_col3, epochs=12, batch_size=25, validation_split=0.2)

Epoch 1/12
2277/2277 [==============================] - 14s 5ms/step - loss: 0.0184 - mse: 0.0184 - val_loss: 0.0117 - val_mse: 0.0117
Epoch 2/12
2277/2277 [==============================] - 13s 6ms/step - loss: 0.0166 - mse: 0.0166 - val_loss: 0.0113 - val_mse: 0.0113
Epoch 3/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0163 - mse: 0.0163 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 4/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0162 - mse: 0.0162 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 5/12
2277/2277 [==============================] - 12s 5ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 6/12
2277/2277 [==============================] - 11s 5ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0112 - val_mse: 0.0112
Epoch 7/12
2277/2277 [==============================] - 12s 5ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0111 - val_mse: 0.0111
Epoch 8/12
2277/2277 [==============================] -

In [ ]:
encoder3 = Model(input3, latent_space3)

In [ ]:
test_au_features3 = encoder3.predict(df_autoencode_col3)

2224/2224 [==============================] - 3s 1ms/step


In [ ]:
test_au_features3

array([[-0.11534631, -0.3389883 ,  0.15642564, ...,  0.49301997,
        -0.12165918, -0.01820565],
       [-0.15070093, -0.30129206,  0.2468383 , ...,  0.51112145,
        -0.08758397, -0.15562579],
       [ 0.02903151, -0.07631189,  0.09185565, ...,  0.5005371 ,
        -0.15091577,  0.02432825],
       ...,
       [ 0.00218086, -0.20417209,  0.09643815, ...,  0.45417267,
         0.08007134,  0.19850677],
       [-0.14881882, -0.18804772, -0.014683  , ...,  0.2732246 ,
        -0.05081696,  0.14954413],
       [-0.06459983, -0.23630565,  0.06509091, ...,  0.32532412,
         0.02972851, -0.19824398]], dtype=float32)

autoencode the autoencoder

In [ ]:
dataframe_enc1=pd.DataFrame(test_au_features1, columns=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v'])
dataframe_enc1 #combine featres into a new df

,a,b,c,d,e,f,g,h,i,j,...,m,n,o,p,q,r,s,t,u,v
0,-0.056439,-0.061521,-0.028192,-0.278656,0.106105,0.400217,-0.152112,0.123114,0.085507,0.385852,...,-0.152314,-0.218254,0.397449,0.099216,0.051740,0.091273,-0.087287,0.256807,-0.063285,0.213141
1,0.044919,-0.148187,-0.064622,-0.168157,-0.038246,0.362593,0.040905,-0.027218,-0.184326,0.301032,...,-0.032623,0.148361,0.416607,0.139224,-0.126089,0.049070,-0.163438,0.104781,-0.260853,0.093776
2,0.022294,-0.117347,-0.073280,-0.044560,-0.042138,0.386314,-0.064505,-0.060137,0.153031,0.392519,...,-0.087658,-0.022297,0.369547,0.299443,-0.035613,0.126062,-0.024230,0.016425,0.015625,0.193440
3,-0.114840,0.119379,-0.471933,0.015203,0.214068,0.204945,0.039410,-0.309551,0.025484,0.325804,...,-0.077746,0.476659,0.586909,0.122257,-0.362353,0.051114,0.025207,-0.268984,0.002338,-0.196375
4,0.191817,-0.354591,-0.335119,0.030119,0.327865,0.427276,0.179710,0.089506,-0.004408,0.265558,...,-0.094837,0.258951,0.403367,0.298742,0.035522,0.003231,0.076559,-0.178588,0.116288,0.068297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,-0.027677,0.006661,-0.169858,0.037544,0.025892,0.387105,-0.037947,0.010258,-0.036322,0.326328,...,-0.090587,-0.043155,0.403263,0.227027,0.038050,0.048798,0.013029,0.098655,-0.026506,0.086683
71147,-0.064759,0.109346,-0.193751,-0.000127,0.003126,0.370450,-0.037107,-0.012482,-0.069149,0.308442,...,-0.064110,0.014902,0.425696,0.226542,-0.025055,-0.066090,-0.009017,0.087765,-0.017535,0.109932
71148,0.057733,0.274948,-0.288644,-0.033510,-0.120637,0.464823,0.165022,0.064155,0.066200,0.262962,...,-0.079992,-0.302665,0.316883,0.275732,-0.403017,0.057724,-0.075586,0.122094,0.000757,0.289392
71149,0.042075,0.124218,-0.227639,0.122592,-0.086572,0.427090,0.115962,0.050723,0.072120,0.255599,...,-0.189727,0.030662,0.377492,0.201152,0.046939,-0.021406,-0.043738,-0.108251,0.088888,0.075153


In [ ]:
dataframe_enc2=pd.DataFrame(test_au_features2, columns=['aa','bb','cc','dd','ee','ff','gg','hh','ii','jj','kk','ll','mm','nn','oo','pp','qq','rr','ss','tt','uu','vv'])
dataframe_enc2

,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj,...,mm,nn,oo,pp,qq,rr,ss,tt,uu,vv
0,-0.225617,0.163912,-0.261736,-0.146426,0.312376,0.290109,0.157598,0.123093,-0.009533,-0.122172,...,-0.577140,-0.078096,-0.384644,-0.028065,0.093894,0.022126,0.142255,-0.065004,0.205767,0.152856
1,-0.007971,0.247512,-0.414137,-0.014328,-0.348266,0.197571,-0.413311,0.132775,-0.083844,-0.029271,...,-0.215017,0.358052,-0.328673,0.144587,0.165849,0.012570,0.083854,-0.040604,-0.098579,0.182798
2,-0.132204,0.134921,-0.158282,-0.039141,0.048281,-0.079578,0.023210,0.135255,-0.112825,0.021998,...,-0.547771,0.008355,-0.421539,0.041636,-0.091901,0.078355,-0.175977,0.140460,0.005574,-0.047307
3,-0.157716,0.060605,0.028469,0.152854,-0.151401,-0.246933,0.242661,0.161193,0.034506,0.045934,...,-0.333683,0.146752,-0.466348,-0.240977,-0.240931,0.285605,-0.190550,-0.076557,-0.334702,0.087996
4,-0.108098,-0.138407,-0.182772,0.053325,-0.304680,0.076545,-0.061523,0.285377,0.030645,0.128807,...,-0.322859,-0.038601,-0.491500,0.049972,0.069769,0.330157,-0.191872,0.060594,0.024431,-0.173509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,-0.144537,0.043911,-0.263531,0.032429,0.035540,0.020147,-0.068143,0.202251,0.006231,-0.050776,...,-0.447206,0.041055,-0.385121,0.052898,-0.002577,0.205068,-0.080425,-0.026276,0.049252,0.070113
71147,-0.107234,0.007047,-0.232377,0.114423,-0.085355,0.001421,-0.039522,0.275644,-0.116986,0.066024,...,-0.503051,0.024111,-0.369911,0.018374,0.015806,0.164796,-0.105868,-0.022319,0.101544,0.105913
71148,-0.177503,-0.099598,-0.287673,-0.067572,0.149810,0.026307,0.013859,0.270218,-0.141279,-0.183721,...,-0.561873,-0.089449,-0.503705,0.020798,-0.182734,0.062449,-0.098950,0.109751,0.190084,0.060297
71149,-0.126587,-0.248618,-0.073166,-0.023270,0.001403,-0.238012,0.036787,0.325795,0.091136,-0.182415,...,-0.501592,0.119402,-0.440260,0.184456,0.105531,0.226833,0.023821,-0.024383,-0.085875,0.043255


In [ ]:
dataframe_enc3=pd.DataFrame(test_au_features3, columns=['a3','b3','c3','d3','e3','f3','g3','h3','i3','j3','k3','l3','m3','n3','o3','p3','q3','r3','s3','t3','u3','v3'])
dataframe_enc3

,a3,b3,c3,d3,e3,f3,g3,h3,i3,j3,...,m3,n3,o3,p3,q3,r3,s3,t3,u3,v3
0,-0.115346,-0.338988,0.156426,-0.110636,0.341002,-0.053351,-0.133411,-0.064276,-0.324274,0.011757,...,0.055331,0.314909,0.117312,0.063491,0.256654,-0.095197,0.000195,0.493020,-0.121659,-0.018206
1,-0.150701,-0.301292,0.246838,-0.083090,0.274802,-0.086980,0.084142,-0.209647,-0.168109,0.067145,...,-0.123284,0.025910,0.150675,0.151547,0.163794,0.009698,0.096063,0.511121,-0.087584,-0.155626
2,0.029032,-0.076312,0.091856,0.086387,0.310979,0.082409,-0.312155,0.061262,-0.313362,-0.103897,...,0.067456,-0.063026,0.061493,0.034945,0.078354,0.083590,-0.190265,0.500537,-0.150916,0.024328
3,-0.191761,-0.218133,0.115132,-0.159631,0.375740,-0.133802,-0.016353,0.004170,-0.418017,0.283300,...,0.198522,-0.142021,0.120274,0.168364,-0.082207,-0.037486,0.160500,0.349802,-0.208607,0.360362
4,-0.245159,-0.194144,0.146370,0.112837,0.257724,-0.094937,-0.206801,0.106121,-0.462559,-0.020348,...,0.090998,0.031422,0.079677,0.108330,0.102331,-0.174342,-0.115800,0.284339,-0.165929,0.051023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,-0.006550,-0.194281,0.115111,-0.057875,0.310034,0.039748,0.071090,0.028289,-0.398630,0.064365,...,0.009412,-0.014438,-0.004773,0.198186,-0.028734,-0.041461,-0.072782,0.408211,-0.017968,-0.008835
71147,-0.067295,-0.199804,0.076565,0.018515,0.334751,0.100448,0.052940,-0.004981,-0.383370,-0.009097,...,-0.009133,-0.021964,0.017245,0.105135,-0.006878,0.049499,-0.178683,0.416118,0.007586,-0.074873
71148,0.002181,-0.204172,0.096438,-0.242781,0.431942,0.223315,0.299780,-0.113004,-0.342163,0.078074,...,-0.105426,0.130277,0.062392,-0.018809,-0.008761,-0.200252,-0.174725,0.454173,0.080071,0.198507
71149,-0.148819,-0.188048,-0.014683,0.055114,0.390900,-0.177104,0.165242,0.170105,-0.474206,0.048437,...,0.030974,0.127129,0.086677,0.144316,0.170046,0.000528,-0.170724,0.273225,-0.050817,0.149544


In [ ]:
df_encoder_combined = pd.concat([dataframe_enc1, dataframe_enc2,dataframe_enc3], axis=1)
df_encoder_combined #take all the encoded values into a new df

,a,b,c,d,e,f,g,h,i,j,...,m3,n3,o3,p3,q3,r3,s3,t3,u3,v3
0,-0.056439,-0.061521,-0.028192,-0.278656,0.106105,0.400217,-0.152112,0.123114,0.085507,0.385852,...,0.055331,0.314909,0.117312,0.063491,0.256654,-0.095197,0.000195,0.493020,-0.121659,-0.018206
1,0.044919,-0.148187,-0.064622,-0.168157,-0.038246,0.362593,0.040905,-0.027218,-0.184326,0.301032,...,-0.123284,0.025910,0.150675,0.151547,0.163794,0.009698,0.096063,0.511121,-0.087584,-0.155626
2,0.022294,-0.117347,-0.073280,-0.044560,-0.042138,0.386314,-0.064505,-0.060137,0.153031,0.392519,...,0.067456,-0.063026,0.061493,0.034945,0.078354,0.083590,-0.190265,0.500537,-0.150916,0.024328
3,-0.114840,0.119379,-0.471933,0.015203,0.214068,0.204945,0.039410,-0.309551,0.025484,0.325804,...,0.198522,-0.142021,0.120274,0.168364,-0.082207,-0.037486,0.160500,0.349802,-0.208607,0.360362
4,0.191817,-0.354591,-0.335119,0.030119,0.327865,0.427276,0.179710,0.089506,-0.004408,0.265558,...,0.090998,0.031422,0.079677,0.108330,0.102331,-0.174342,-0.115800,0.284339,-0.165929,0.051023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,-0.027677,0.006661,-0.169858,0.037544,0.025892,0.387105,-0.037947,0.010258,-0.036322,0.326328,...,0.009412,-0.014438,-0.004773,0.198186,-0.028734,-0.041461,-0.072782,0.408211,-0.017968,-0.008835
71147,-0.064759,0.109346,-0.193751,-0.000127,0.003126,0.370450,-0.037107,-0.012482,-0.069149,0.308442,...,-0.009133,-0.021964,0.017245,0.105135,-0.006878,0.049499,-0.178683,0.416118,0.007586,-0.074873
71148,0.057733,0.274948,-0.288644,-0.033510,-0.120637,0.464823,0.165022,0.064155,0.066200,0.262962,...,-0.105426,0.130277,0.062392,-0.018809,-0.008761,-0.200252,-0.174725,0.454173,0.080071,0.198507
71149,0.042075,0.124218,-0.227639,0.122592,-0.086572,0.427090,0.115962,0.050723,0.072120,0.255599,...,0.030974,0.127129,0.086677,0.144316,0.170046,0.000528,-0.170724,0.273225,-0.050817,0.149544


In [ ]:
inputFinal = Input(shape=df_encoder_combined.shape[1:])
encoder = Dense(48)(inputFinal)
encoder = LeakyReLU(alpha=0.15)(encoder)
encoder = Dense(32)(encoder)
encoder = LeakyReLU(alpha=0.15)(encoder)
latent_spaceFinal = Dense(16, activation="tanh")(encoder)

decoder = Dense(32)(latent_spaceFinal)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(48)(decoder)
decoder = LeakyReLU(alpha=0.15)(decoder)
decoder = Dense(units=df_encoder_combined.shape[1], activation="tanh")(decoder)

autoencoder = Model(inputFinal, decoder)
autoencoder.compile(optimizer = "adam",metrics=["mse"],loss="mse")
autoencoder.fit(df_encoder_combined, df_encoder_combined, epochs=12, batch_size=25, validation_split=0.22)

Epoch 1/12
2220/2220 [==============================] - 14s 5ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0042 - val_mse: 0.0042
Epoch 2/12
2220/2220 [==============================] - 11s 5ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 3/12
2220/2220 [==============================] - 12s 5ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 4/12
2220/2220 [==============================] - 12s 5ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 5/12
2220/2220 [==============================] - 10s 5ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 6/12
2220/2220 [==============================] - 11s 5ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 7/12
2220/2220 [==============================] - 12s 5ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 8/12
2220/2220 [==============================] -

take the latent space

In [ ]:
encoderFinal = Model(inputFinal, latent_spaceFinal)

In [ ]:
test_au_featuresFinal = encoderFinal.predict(df_encoder_combined)


2224/2224 [==============================] - 3s 1ms/step


In [ ]:
test_au_featuresFinal

array([[-0.05671838,  0.16305043,  0.14446446, ...,  0.03802745,
        -0.01857514, -0.15161422],
       [-0.03358361,  0.07336806,  0.0661437 , ...,  0.14771359,
         0.14490965, -0.06178718],
       [-0.11602808,  0.09648061, -0.02365895, ...,  0.20493539,
         0.00780694, -0.07219609],
       ...,
       [ 0.11650037,  0.3542126 ,  0.04626064, ...,  0.07940051,
        -0.04170864, -0.06586905],
       [ 0.02467952,  0.08718507, -0.12284064, ...,  0.18890673,
        -0.02608765,  0.04200338],
       [-0.05913313,  0.15418427, -0.10759871, ...,  0.18855447,
        -0.0795533 , -0.06473269]], dtype=float32)

In [ ]:
dataframe_encoder_final=pd.DataFrame(test_au_featuresFinal, columns=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p'])
dataframe_encoder_final

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
0,-0.056718,0.163050,0.144464,-0.004514,-0.217304,0.093837,-0.079056,0.048420,-0.109437,0.126497,0.080608,0.091150,0.013630,0.038027,-0.018575,-0.151614
1,-0.033584,0.073368,0.066144,0.187574,0.046512,0.048895,0.029059,-0.049920,0.150401,0.091211,0.211238,-0.080088,-0.105701,0.147714,0.144910,-0.061787
2,-0.116028,0.096481,-0.023659,0.011074,-0.024519,-0.024958,0.085362,-0.062610,-0.033227,-0.040190,-0.016756,0.040291,0.018368,0.204935,0.007807,-0.072196
3,0.089969,-0.014347,-0.144617,0.169569,0.008621,-0.065365,0.030311,0.280329,-0.002046,0.048338,-0.046592,0.114506,0.223265,0.272000,0.137267,-0.096172
4,-0.069115,-0.076028,-0.076041,0.045921,0.176450,-0.031422,0.035509,-0.082954,0.100949,-0.094502,-0.076700,0.003328,-0.065457,0.170309,-0.005865,-0.093412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,-0.017544,0.131473,0.050048,0.034998,-0.047015,-0.050616,-0.048037,-0.047304,-0.037163,-0.009100,0.002045,-0.001011,-0.044459,0.168736,0.013231,-0.060739
71147,-0.006284,0.164392,0.017716,0.076029,0.009437,-0.033199,-0.065322,-0.094455,-0.010365,-0.005671,0.025321,0.014478,-0.071053,0.194353,0.005859,-0.080324
71148,0.116500,0.354213,0.046261,0.137206,-0.218185,-0.008972,0.001405,-0.088531,0.007120,-0.032226,-0.055120,0.056336,-0.020061,0.079401,-0.041709,-0.065869
71149,0.024680,0.087185,-0.122841,0.100630,-0.094843,-0.086207,0.016320,-0.053709,-0.065794,0.015536,0.051248,-0.102428,0.026431,0.188907,-0.026088,0.042003


In [ ]:
dataframe_encoder_final.iloc[[15240]]

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
15240,0.000657,0.131362,0.06011,0.019356,-0.041958,0.04955,0.005169,-0.089387,-0.008709,0.004719,0.107397,-0.120499,-0.023724,0.20394,-0.030835,-0.102909


In [ ]:
df_Only_non_sequential = df_data_better[['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre','Duration','Danceability', 'Energy','Key','Loudness','Speechiness','Acousticness','Instrumentalness','Liveness','Valence','Tempo','TimeSignature','ZeroCrossing','MFCCSMax','MFCCSMin','MFCCSMean','MFCCSMedian','Rms','Bpm','RolloffMin','RolloffMax','ChromaMedian','ChromaMean','ChromaMin','ChromaMax']]
df_Only_non_sequential #jugglint the dataframes so that they have the correct data for training

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,MFCCSMean,MFCCSMedian,Rms,Bpm,RolloffMin,RolloffMax,ChromaMedian,ChromaMean,ChromaMin,ChromaMax
0,37Jdi2cCSRA5jl6zADeLOy,D R O V E MY CAR,1 Trait High,https://p.scdn.co/mp3-preview/ab54f8f789175d42...,1 Trait Danger,comedy rap,120937,0.601,0.827,2,...,5.030483,3.302809,0.304123,128.129013,73.953094,6760.593052,0.349420,0.421962,0.005061,1
1,14jLrcO0NGIxtX56LHEhsv,Reprisal,1 Trait World Tour,https://p.scdn.co/mp3-preview/ffe73947f36e6e15...,1 Trait Danger,comedy rap,100634,0.391,0.575,5,...,3.887536,3.168254,0.285675,72.090485,197.802658,7413.242569,0.478386,0.531803,0.053862,1
2,7dcNDXD9SwJ4Txqg2fE2t8,Danger,Back from the Dead,https://p.scdn.co/mp3-preview/1ae38bb85e0313f6...,LUCKY3RD,houston rap,181395,0.802,0.511,6,...,1.650047,2.126344,0.297220,84.936028,185.067787,6498.745937,0.329896,0.406566,0.001817,1
3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.569,5,...,5.127454,3.298813,0.405741,120.059700,127.567406,5817.598572,0.521628,0.536370,0.011882,1
4,1wDY9VWdCoRjLV4gUyYf52,Danger,In Transit (Deluxe),https://p.scdn.co/mp3-preview/61586296e9713d07...,Terri,nigerian pop,157648,0.514,0.582,11,...,1.093476,-0.297896,0.299387,97.207802,141.000423,5296.091309,0.248856,0.346003,0.001743,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,75dwXgnlK10c8YtvUODVbj,My Foolish Heart,Everything I Have Is Yours / The Best Of The M...,https://p.scdn.co/mp3-preview/15f2927aa3ef44ae...,Billy Eckstine,adult standards,185906,0.316,0.110,9,...,-5.347975,-3.515098,0.096891,96.391342,218.175087,2585.977879,0.069974,0.215599,0.000245,1
71147,4f9Zf8gkeZ5NuSRyElXa7r,Alalaga,Samoa With Love,https://p.scdn.co/mp3-preview/df69922b04c63104...,The Five Stars,pacific islands pop,518666,0.355,0.362,1,...,-2.967051,-3.451842,0.147721,85.702225,155.476456,3527.954578,0.191280,0.296154,0.001543,1
71148,3e4OUcQE9tTujfgT0MbKq0,Feel The Fire - Futuristic Polar Bears & Jerry...,Feel The Fire (The Remixes),https://p.scdn.co/mp3-preview/9c6fb71eaac48cca...,MR.BLACK,big room,161904,0.591,0.973,1,...,3.457322,0.285472,0.333917,126.038162,242.130775,7507.046585,0.433053,0.488847,0.041757,1
71149,2nnMT7bf8F5xBYOBzZiDZC,Blast Off To Nowhere,Tonight The Stars Revolt,https://p.scdn.co/mp3-preview/221e0fa36c554556...,Powerman 5000,alternative metal,225293,0.522,0.988,1,...,2.860447,-0.272149,0.282717,142.889572,220.967674,6753.830280,0.480341,0.522678,0.039996,1


In [ ]:
df_non_seq_and_autoencoded = pd.concat([df_Only_non_sequential, dataframe_encoder_final], axis=1) #this is df_better_data with encoded values 
df_non_seq_and_autoencoded

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,g,h,i,j,k,l,m,n,o,p
0,37Jdi2cCSRA5jl6zADeLOy,D R O V E MY CAR,1 Trait High,https://p.scdn.co/mp3-preview/ab54f8f789175d42...,1 Trait Danger,comedy rap,120937,0.601,0.827,2,...,-0.079056,0.048420,-0.109437,0.126497,0.080608,0.091150,0.013630,0.038027,-0.018575,-0.151614
1,14jLrcO0NGIxtX56LHEhsv,Reprisal,1 Trait World Tour,https://p.scdn.co/mp3-preview/ffe73947f36e6e15...,1 Trait Danger,comedy rap,100634,0.391,0.575,5,...,0.029059,-0.049920,0.150401,0.091211,0.211238,-0.080088,-0.105701,0.147714,0.144910,-0.061787
2,7dcNDXD9SwJ4Txqg2fE2t8,Danger,Back from the Dead,https://p.scdn.co/mp3-preview/1ae38bb85e0313f6...,LUCKY3RD,houston rap,181395,0.802,0.511,6,...,0.085362,-0.062610,-0.033227,-0.040190,-0.016756,0.040291,0.018368,0.204935,0.007807,-0.072196
3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.569,5,...,0.030311,0.280329,-0.002046,0.048338,-0.046592,0.114506,0.223265,0.272000,0.137267,-0.096172
4,1wDY9VWdCoRjLV4gUyYf52,Danger,In Transit (Deluxe),https://p.scdn.co/mp3-preview/61586296e9713d07...,Terri,nigerian pop,157648,0.514,0.582,11,...,0.035509,-0.082954,0.100949,-0.094502,-0.076700,0.003328,-0.065457,0.170309,-0.005865,-0.093412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,75dwXgnlK10c8YtvUODVbj,My Foolish Heart,Everything I Have Is Yours / The Best Of The M...,https://p.scdn.co/mp3-preview/15f2927aa3ef44ae...,Billy Eckstine,adult standards,185906,0.316,0.110,9,...,-0.048037,-0.047304,-0.037163,-0.009100,0.002045,-0.001011,-0.044459,0.168736,0.013231,-0.060739
71147,4f9Zf8gkeZ5NuSRyElXa7r,Alalaga,Samoa With Love,https://p.scdn.co/mp3-preview/df69922b04c63104...,The Five Stars,pacific islands pop,518666,0.355,0.362,1,...,-0.065322,-0.094455,-0.010365,-0.005671,0.025321,0.014478,-0.071053,0.194353,0.005859,-0.080324
71148,3e4OUcQE9tTujfgT0MbKq0,Feel The Fire - Futuristic Polar Bears & Jerry...,Feel The Fire (The Remixes),https://p.scdn.co/mp3-preview/9c6fb71eaac48cca...,MR.BLACK,big room,161904,0.591,0.973,1,...,0.001405,-0.088531,0.007120,-0.032226,-0.055120,0.056336,-0.020061,0.079401,-0.041709,-0.065869
71149,2nnMT7bf8F5xBYOBzZiDZC,Blast Off To Nowhere,Tonight The Stars Revolt,https://p.scdn.co/mp3-preview/221e0fa36c554556...,Powerman 5000,alternative metal,225293,0.522,0.988,1,...,0.016320,-0.053709,-0.065794,0.015536,0.051248,-0.102428,0.026431,0.188907,-0.026088,0.042003


In [ ]:
df_first_cluster = df_non_seq_and_autoencoded.copy()

In [ ]:
df_first_cluster = df_first_cluster.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre','Duration'], axis=1) #drop these for training on som
df_first_cluster

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,...,g,h,i,j,k,l,m,n,o,p
0,0.601,0.827,2,-7.056,0.1350,0.01180,0.000000,0.9610,0.2930,128.055,...,-0.079056,0.048420,-0.109437,0.126497,0.080608,0.091150,0.013630,0.038027,-0.018575,-0.151614
1,0.391,0.575,5,-7.520,0.1070,0.38000,0.004660,0.4130,0.3260,143.260,...,0.029059,-0.049920,0.150401,0.091211,0.211238,-0.080088,-0.105701,0.147714,0.144910,-0.061787
2,0.802,0.511,6,-9.651,0.2980,0.03120,0.000000,0.0843,0.3320,84.979,...,0.085362,-0.062610,-0.033227,-0.040190,-0.016756,0.040291,0.018368,0.204935,0.007807,-0.072196
3,0.720,0.569,5,-6.983,0.0607,0.01650,0.000525,0.5400,0.1690,119.989,...,0.030311,0.280329,-0.002046,0.048338,-0.046592,0.114506,0.223265,0.272000,0.137267,-0.096172
4,0.514,0.582,11,-5.243,0.3880,0.35500,0.000001,0.1840,0.4530,66.681,...,0.035509,-0.082954,0.100949,-0.094502,-0.076700,0.003328,-0.065457,0.170309,-0.005865,-0.093412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,0.316,0.110,9,-17.251,0.0367,0.99200,0.539000,0.4030,0.1220,88.791,...,-0.048037,-0.047304,-0.037163,-0.009100,0.002045,-0.001011,-0.044459,0.168736,0.013231,-0.060739
71147,0.355,0.362,1,-13.699,0.0340,0.70900,0.623000,0.1220,0.3570,86.946,...,-0.065322,-0.094455,-0.010365,-0.005671,0.025321,0.014478,-0.071053,0.194353,0.005859,-0.080324
71148,0.591,0.973,1,-2.835,0.0471,0.00420,0.390000,0.1850,0.0419,125.974,...,0.001405,-0.088531,0.007120,-0.032226,-0.055120,0.056336,-0.020061,0.079401,-0.041709,-0.065869
71149,0.522,0.988,1,-4.646,0.3210,0.00320,0.051300,0.1200,0.0689,141.685,...,0.016320,-0.053709,-0.065794,0.015536,0.051248,-0.102428,0.026431,0.188907,-0.026088,0.042003


In [ ]:
df_first_cluster=sc.fit_transform(df_first_cluster)
df_first_cluster

array([[0.60953347, 0.827     , 0.18181818, ..., 0.51535765, 0.30133178,
        0.35789274],
       [0.39655172, 0.575     , 0.45454545, ..., 0.63877592, 0.46225689,
        0.49264401],
       [0.81338742, 0.511     , 0.54545455, ..., 0.70316159, 0.3273008 ,
        0.4770294 ],
       ...,
       [0.59939148, 0.973     , 0.09090909, ..., 0.5619104 , 0.27856049,
        0.48652072],
       [0.52941176, 0.988     , 0.09090909, ..., 0.68512623, 0.2939369 ,
        0.64834221],
       [0.94827586, 0.484     , 0.45454545, ..., 0.68472987, 0.24130837,
        0.4882254 ]])

In [ ]:
songsom = SOM(m=11, n=4, dim=40) #create SOM with 44 nodes
songsom.fit(df_first_cluster) #fit som on values

In [ ]:
predictions = songsom.predict(df_first_cluster) #cluster predictions
predictions


array([16, 17, 19, ..., 26, 26, 39])

In [ ]:
max(predictions)

43

In [ ]:
len(np.unique(predictions))


44

In [ ]:
#df_data_better = pd.DataFrame(test_au_features)
df_non_seq_and_autoencoded.insert(47, "clusterLabels", predictions, True) #insert at 47 in actual run

In [ ]:
df_non_seq_and_autoencoded

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,h,i,j,k,l,m,n,o,p,clusterLabels
0,37Jdi2cCSRA5jl6zADeLOy,D R O V E MY CAR,1 Trait High,https://p.scdn.co/mp3-preview/ab54f8f789175d42...,1 Trait Danger,comedy rap,120937,0.601,0.827,2,...,0.048420,-0.109437,0.126497,0.080608,0.091150,0.013630,0.038027,-0.018575,-0.151614,16
1,14jLrcO0NGIxtX56LHEhsv,Reprisal,1 Trait World Tour,https://p.scdn.co/mp3-preview/ffe73947f36e6e15...,1 Trait Danger,comedy rap,100634,0.391,0.575,5,...,-0.049920,0.150401,0.091211,0.211238,-0.080088,-0.105701,0.147714,0.144910,-0.061787,17
2,7dcNDXD9SwJ4Txqg2fE2t8,Danger,Back from the Dead,https://p.scdn.co/mp3-preview/1ae38bb85e0313f6...,LUCKY3RD,houston rap,181395,0.802,0.511,6,...,-0.062610,-0.033227,-0.040190,-0.016756,0.040291,0.018368,0.204935,0.007807,-0.072196,19
3,0ackjn8zBcLQnkeRPeOCVj,Danger,Danger,https://p.scdn.co/mp3-preview/ab856e3e92deddc1...,Cooliecut,dark trap,96052,0.720,0.569,5,...,0.280329,-0.002046,0.048338,-0.046592,0.114506,0.223265,0.272000,0.137267,-0.096172,23
4,1wDY9VWdCoRjLV4gUyYf52,Danger,In Transit (Deluxe),https://p.scdn.co/mp3-preview/61586296e9713d07...,Terri,nigerian pop,157648,0.514,0.582,11,...,-0.082954,0.100949,-0.094502,-0.076700,0.003328,-0.065457,0.170309,-0.005865,-0.093412,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71146,75dwXgnlK10c8YtvUODVbj,My Foolish Heart,Everything I Have Is Yours / The Best Of The M...,https://p.scdn.co/mp3-preview/15f2927aa3ef44ae...,Billy Eckstine,adult standards,185906,0.316,0.110,9,...,-0.047304,-0.037163,-0.009100,0.002045,-0.001011,-0.044459,0.168736,0.013231,-0.060739,1
71147,4f9Zf8gkeZ5NuSRyElXa7r,Alalaga,Samoa With Love,https://p.scdn.co/mp3-preview/df69922b04c63104...,The Five Stars,pacific islands pop,518666,0.355,0.362,1,...,-0.094455,-0.010365,-0.005671,0.025321,0.014478,-0.071053,0.194353,0.005859,-0.080324,4
71148,3e4OUcQE9tTujfgT0MbKq0,Feel The Fire - Futuristic Polar Bears & Jerry...,Feel The Fire (The Remixes),https://p.scdn.co/mp3-preview/9c6fb71eaac48cca...,MR.BLACK,big room,161904,0.591,0.973,1,...,-0.088531,0.007120,-0.032226,-0.055120,0.056336,-0.020061,0.079401,-0.041709,-0.065869,26
71149,2nnMT7bf8F5xBYOBzZiDZC,Blast Off To Nowhere,Tonight The Stars Revolt,https://p.scdn.co/mp3-preview/221e0fa36c554556...,Powerman 5000,alternative metal,225293,0.522,0.988,1,...,-0.053709,-0.065794,0.015536,0.051248,-0.102428,0.026431,0.188907,-0.026088,0.042003,26


In [ ]:
df_clust0 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 0] #this next section turns each cluster into a dataframe
len(df_clust0)

2157

In [ ]:
df_clust1 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 1]
len(df_clust1)

1880

In [ ]:
df_clust2 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 2]
len(df_clust2)

1039

In [ ]:
df_clust3 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 3]
len(df_clust3)

974

In [ ]:
df_clust4 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 4]
len(df_clust4)

2190

In [ ]:
df_clust5 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 5]
len(df_clust5)

358

In [ ]:
df_clust6 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 6]
len(df_clust6)

847

In [ ]:
df_clust7 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 7]
len(df_clust7)

1567

In [ ]:
df_clust8 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 8]
len(df_clust8)

423

In [ ]:
df_clust9 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 9]
len(df_clust9)

2345

In [ ]:
df_clust10 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 10]
len(df_clust10)

823

In [ ]:
df_clust11 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 11]
len(df_clust11)

1547

In [ ]:
df_clust12 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 12]
len(df_clust12)

1399

In [ ]:
df_clust13 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 13]
len(df_clust13)

2408

In [ ]:
df_clust14 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 14]
len(df_clust14)

1586

In [ ]:
df_clust15 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 15]
len(df_clust15)

2061

In [ ]:
df_clust16 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 16]
len(df_clust16)

1997

In [ ]:
df_clust17 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 17]
len(df_clust17)

576

In [ ]:
df_clust18 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 18]
len(df_clust18)

948

In [ ]:
df_clust19 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 19]


In [ ]:
df_clust20 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 20]


In [ ]:
df_clust21 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 21]


In [ ]:
df_clust22 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 22]


In [ ]:
df_clust23 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 23]


In [ ]:
df_clust24 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 24]


In [ ]:
df_clust25 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 25]


In [ ]:
df_clust26 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 26]


In [ ]:
df_clust27 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 27]


In [ ]:
df_clust28 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 28]


In [ ]:
df_clust29 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 29]


In [ ]:
df_clust30 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 30]


In [ ]:
df_clust31 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 31]


In [ ]:
df_clust32 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 32]


In [ ]:
df_clust33 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 33]


In [ ]:
df_clust34 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 34]


In [ ]:
df_clust35 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 35]


In [ ]:
df_clust36 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 36]


In [ ]:
df_clust37 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 37]


In [ ]:
df_clust38 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 38]
len(df_clust38)

284

In [ ]:
df_clust39 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 39]
len(df_clust39)

1676

In [ ]:
df_clust40 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 40]
len(df_clust40)

2722

In [ ]:
df_clust42 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 42]
len(df_clust42)

2269

In [ ]:
df_clust41 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 41]
len(df_clust41)

560

In [ ]:
df_clust43 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 43]
len(df_clust43)

2786

In [ ]:
df_clust44 = df_non_seq_and_autoencoded.loc[df_non_seq_and_autoencoded['clusterLabels'] == 44] 
len(df_clust44)

0

Make copies and redrop values

In [ ]:
df_clust44_big = df_clust44.copy() #make copies to redrop values
df_clust0_big = df_clust0.copy()
df_clust1_big = df_clust1.copy()
df_clust2_big = df_clust2.copy()
df_clust3_big = df_clust3.copy()
df_clust4_big = df_clust4.copy()
df_clust5_big = df_clust5.copy()
df_clust6_big = df_clust6.copy()
df_clust7_big = df_clust7.copy()
df_clust8_big = df_clust8.copy()
df_clust9_big = df_clust9.copy()
df_clust10_big = df_clust10.copy()
df_clust11_big = df_clust11.copy()
df_clust12_big = df_clust12.copy()
df_clust13_big = df_clust13.copy()
df_clust14_big = df_clust14.copy()
df_clust15_big = df_clust15.copy()
df_clust16_big = df_clust16.copy()
df_clust17_big = df_clust17.copy()
df_clust18_big = df_clust18.copy()
df_clust19_big = df_clust19.copy()
df_clust20_big = df_clust20.copy()
df_clust21_big = df_clust21.copy()
df_clust22_big = df_clust22.copy()
df_clust23_big = df_clust23.copy()
df_clust24_big = df_clust24.copy()
df_clust25_big = df_clust25.copy()
df_clust26_big = df_clust26.copy()
df_clust27_big = df_clust27.copy()
df_clust28_big = df_clust28.copy()
df_clust29_big = df_clust29.copy()
df_clust30_big = df_clust30.copy()
df_clust31_big = df_clust31.copy()
df_clust32_big = df_clust32.copy()
df_clust33_big = df_clust33.copy()
df_clust34_big = df_clust34.copy()
df_clust35_big = df_clust35.copy()
df_clust36_big = df_clust36.copy()
df_clust37_big = df_clust37.copy()
df_clust38_big = df_clust38.copy()
df_clust39_big = df_clust39.copy()
df_clust40_big = df_clust40.copy()
df_clust41_big = df_clust41.copy()
df_clust42_big = df_clust42.copy()
df_clust43_big = df_clust43.copy()

In [ ]:
df_clust0 = df_clust0.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1) #drop values which cant be trained 
df_clust1 = df_clust1.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust2 = df_clust2.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust3 = df_clust3.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust4 = df_clust4.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust5 = df_clust5.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust6 = df_clust6.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust7 = df_clust7.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust8 = df_clust8.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust9 = df_clust9.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust10 = df_clust10.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust11 = df_clust11.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust12 = df_clust12.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust13 = df_clust13.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust14 = df_clust14.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust15 = df_clust15.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust16 = df_clust16.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust17 = df_clust17.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust18 = df_clust18.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust19 = df_clust19.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust20 = df_clust20.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust21 = df_clust21.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust22 = df_clust22.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust23 = df_clust23.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust24 = df_clust24.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust25 = df_clust25.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust26 = df_clust26.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust27 = df_clust27.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust28 = df_clust28.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust29 = df_clust29.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust30 = df_clust30.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust31 = df_clust31.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust32 = df_clust32.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust33 = df_clust33.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust34 = df_clust34.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust35 = df_clust35.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust36 = df_clust36.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust37 = df_clust37.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust38 = df_clust38.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust39 = df_clust39.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust40 = df_clust40.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust41 = df_clust41.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust42 = df_clust42.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)
df_clust43 = df_clust43.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)

In [ ]:
df_clust44 = df_clust44.drop(['TrackID', 'TrackName', 'Album', 'PreviewURL', 'ArtistName', 'Genre'], axis=1)


In [ ]:
df_clust44

,Duration,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,...,h,i,j,k,l,m,n,o,p,clusterLabels


distance calculation stuff

## Clustering the clusters


In [ ]:

#df_clust44=sc.fit_transform(df_clust44)



In [ ]:
df_clust0=sc.fit_transform(df_clust0)#do cluster calculations on each of the bigger clusters to make subclusters
df_clust0
df_clust1=sc.fit_transform(df_clust1)
df_clust1
df_clust2=sc.fit_transform(df_clust2)
df_clust2
df_clust3=sc.fit_transform(df_clust3)
df_clust3
df_clust4=sc.fit_transform(df_clust4)
df_clust4
df_clust5=sc.fit_transform(df_clust5)
df_clust5
df_clust6=sc.fit_transform(df_clust6)
df_clust6
df_clust7=sc.fit_transform(df_clust7)
df_clust7

array([[0.02883891, 0.73677069, 0.29943398, ..., 0.61524252, 0.39259819,
        0.        ],
       [0.04098408, 0.68113976, 0.30175373, ..., 0.44690238, 0.34248515,
        0.        ],
       [0.01316059, 0.69742198, 0.33770994, ..., 0.58965798, 0.44032899,
        0.        ],
       ...,
       [0.07276764, 0.10719132, 0.55228728, ..., 0.79339636, 0.46545331,
        0.        ],
       [0.05238325, 0.86431479, 0.32843092, ..., 0.48533637, 0.42082073,
        0.        ],
       [0.03460455, 0.4843962 , 0.35278835, ..., 0.57572321, 0.36828041,
        0.        ]])

In [ ]:
df_clust8=sc.fit_transform(df_clust8)
df_clust8
df_clust9=sc.fit_transform(df_clust9)
df_clust9
df_clust10=sc.fit_transform(df_clust10)
df_clust10
df_clust11=sc.fit_transform(df_clust11)
df_clust11
df_clust12=sc.fit_transform(df_clust12)
df_clust12
df_clust13=sc.fit_transform(df_clust13)
df_clust13
df_clust14=sc.fit_transform(df_clust14)
df_clust14
df_clust15=sc.fit_transform(df_clust15)
df_clust15
df_clust16=sc.fit_transform(df_clust16)
df_clust16
df_clust17=sc.fit_transform(df_clust17)
df_clust17
df_clust18=sc.fit_transform(df_clust18)
df_clust18
df_clust19=sc.fit_transform(df_clust19)
df_clust19
df_clust20=sc.fit_transform(df_clust20)
df_clust20
df_clust21=sc.fit_transform(df_clust21)
df_clust21
df_clust22=sc.fit_transform(df_clust22)
df_clust22
df_clust23=sc.fit_transform(df_clust23)
df_clust23
df_clust24=sc.fit_transform(df_clust24)
df_clust24
df_clust25=sc.fit_transform(df_clust25)
df_clust25
df_clust26=sc.fit_transform(df_clust26)
df_clust26
df_clust27=sc.fit_transform(df_clust27)
df_clust27
df_clust28=sc.fit_transform(df_clust28)
df_clust28
df_clust29=sc.fit_transform(df_clust29)
df_clust29
df_clust30=sc.fit_transform(df_clust30)
df_clust30
df_clust31=sc.fit_transform(df_clust31)
df_clust31
df_clust32=sc.fit_transform(df_clust32)
df_clust32
df_clust33=sc.fit_transform(df_clust33)
df_clust33
df_clust34=sc.fit_transform(df_clust34)
df_clust34
df_clust35=sc.fit_transform(df_clust35)
df_clust35
df_clust36=sc.fit_transform(df_clust36)
df_clust36
df_clust37=sc.fit_transform(df_clust37)
df_clust37


array([[0.11454328, 0.88652482, 0.81025641, ..., 0.57079148, 0.41991135,
        0.        ],
       [0.02106831, 0.66950355, 0.43717949, ..., 0.43667643, 0.37655635,
        0.        ],
       [0.25105733, 0.90212766, 0.83846154, ..., 0.55505187, 0.52599722,
        0.        ],
       ...,
       [0.2912187 , 0.74042553, 0.66794872, ..., 0.44933939, 0.33467004,
        0.        ],
       [0.44571594, 0.59716312, 0.36794872, ..., 0.50698938, 0.41432351,
        0.        ],
       [0.18127836, 0.54468085, 0.47307692, ..., 0.51456288, 0.50537596,
        0.        ]])

In [ ]:
df_clust38=sc.fit_transform(df_clust38)
df_clust38
df_clust39=sc.fit_transform(df_clust39)
df_clust39
df_clust40=sc.fit_transform(df_clust40)
df_clust40
df_clust41=sc.fit_transform(df_clust41)
df_clust41
df_clust42=sc.fit_transform(df_clust42)
df_clust42
df_clust43=sc.fit_transform(df_clust43)
df_clust43

array([[0.0637308 , 0.3003096 , 0.579974  , ..., 0.16281062, 0.59499249,
        0.        ],
       [0.02449394, 0.49845201, 0.62497679, ..., 0.64264777, 0.41393863,
        0.        ],
       [0.30350714, 0.30237358, 0.78698681, ..., 0.32918911, 0.44151723,
        0.        ],
       ...,
       [0.04869438, 0.24355005, 0.84599047, ..., 0.43349968, 0.36768256,
        0.        ],
       [0.14673131, 0.53869969, 0.94399653, ..., 0.47145994, 0.28949846,
        0.        ],
       [0.02421287, 0.38080495, 0.32095797, ..., 0.54459607, 0.46148821,
        0.        ]])

predictions for each 

In [ ]:
songsom = SOM(m=3, n=6, dim=42)#recluster the values 
#songsom.fit(df_clust44)


In [ ]:
songsom.fit(df_clust0)
songsom.fit(df_clust1)
songsom.fit(df_clust2)
songsom.fit(df_clust3)
songsom.fit(df_clust4)
songsom.fit(df_clust5)
songsom.fit(df_clust6)
songsom.fit(df_clust7)
songsom.fit(df_clust8)
songsom.fit(df_clust9)
songsom.fit(df_clust10)
songsom.fit(df_clust11)
songsom.fit(df_clust12)
songsom.fit(df_clust13)
songsom.fit(df_clust14)
songsom.fit(df_clust15)
songsom.fit(df_clust16)
songsom.fit(df_clust17)
songsom.fit(df_clust18)
songsom.fit(df_clust19)
songsom.fit(df_clust20)
songsom.fit(df_clust21)
songsom.fit(df_clust22)
songsom.fit(df_clust23)
songsom.fit(df_clust24)
songsom.fit(df_clust25)
songsom.fit(df_clust26)
songsom.fit(df_clust27)
songsom.fit(df_clust28)
songsom.fit(df_clust29)
songsom.fit(df_clust30)
songsom.fit(df_clust31)
songsom.fit(df_clust32)
songsom.fit(df_clust33)
songsom.fit(df_clust34)
songsom.fit(df_clust35)
songsom.fit(df_clust36)
songsom.fit(df_clust37)
songsom.fit(df_clust38)
songsom.fit(df_clust39)
songsom.fit(df_clust40)
songsom.fit(df_clust41)
songsom.fit(df_clust42)
songsom.fit(df_clust43)
#songsom.fit(df_clust44)

In [ ]:
predictions = songsom.predict(df_clust0)
df_clust0_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust1)
df_clust1_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust2)
df_clust2_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust3)
df_clust3_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust4)
df_clust4_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust5)
df_clust5_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust6)
df_clust6_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust7)
df_clust7_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust8)
df_clust8_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust9)
df_clust9_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust10)
df_clust10_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust11)
df_clust11_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust12)
df_clust12_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust13)
df_clust13_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust14)
df_clust14_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust15)
df_clust15_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust16)
df_clust16_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust17)
df_clust17_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust18)
df_clust18_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust19)
df_clust19_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust20)
df_clust20_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust21)
df_clust21_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust22)
df_clust22_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust23)
df_clust23_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust24)
df_clust24_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust25)
df_clust25_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust26)
df_clust26_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust27)
df_clust27_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust28)
df_clust28_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust29)
df_clust29_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust30)
df_clust30_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust31)
df_clust31_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust32)
df_clust32_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust33)
df_clust33_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust34)
df_clust34_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust35)
df_clust35_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust36)
df_clust36_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust37)
df_clust37_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust38)
df_clust38_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust39)
df_clust39_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust40)
df_clust40_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust41)
df_clust41_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust42)
df_clust42_big.insert(48, "clusterLabelsSecond", predictions, True)
predictions = songsom.predict(df_clust43)
df_clust43_big.insert(48, "clusterLabelsSecond", predictions, True)


In [ ]:
df_clust43_big

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,i,j,k,l,m,n,o,p,clusterLabels,clusterLabelsSecond
5,0g7S3JquNLgjMyIv7vdcaE,7.53,Furi (Original Game Soundtrack),https://p.scdn.co/mp3-preview/afb24681babb9cec...,Danger,darksynth,157377,0.291,0.580,7,...,-0.097356,0.084776,-0.128331,-0.108078,-0.081206,0.125994,-0.186962,-0.004572,43,3
40,5uODHxCoTuirCnk0oEYyYv,19.06,Furi (Original Game Soundtrack),https://p.scdn.co/mp3-preview/700ccce7c7f1e659...,Danger,darksynth,75294,0.483,0.625,0,...,-0.045850,-0.102487,0.138913,0.045962,-0.090276,0.296679,0.083665,-0.090141,43,4
79,0hBby0yygBY1u3m6tSpZgC,The Camera Eye,Moving Pictures (2011 Remaster),https://p.scdn.co/mp3-preview/3b8444df3fa6e7e2...,Rush,album rock,658986,0.293,0.787,5,...,-0.029553,0.039158,0.004074,0.068507,-0.052676,0.206586,-0.093125,-0.077107,43,9
96,4CyElmiM9EF9aURGWlPZLi,La Villa Strangiato,Hemispheres,https://p.scdn.co/mp3-preview/7a4b4080e212e6fd...,Rush,album rock,574733,0.366,0.606,0,...,0.015063,0.066726,0.051960,-0.039492,0.082736,0.139541,-0.029199,-0.061400,43,0
221,1bXhlDbeU61q21LOJWbKN9,Magellan's Expedition for Dogs,Seventh Rum of a Seventh Rum (Deluxe Version),https://p.scdn.co/mp3-preview/f6822fb8341a2900...,Alestorm,british power metal,278899,0.414,0.982,8,...,-0.033862,-0.012645,-0.056331,0.074875,-0.055280,0.203444,-0.110630,-0.134332,43,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71097,704d2awSVVDpY2b8Xn0N8e,Explode - Remix by Dope Stars Inc.,The Perfect Cult,https://p.scdn.co/mp3-preview/33b848af89c6799e...,Deathstars,cyber metal,331013,0.522,0.944,1,...,0.058544,0.047814,-0.153424,0.020373,0.022648,0.121276,-0.012885,-0.148953,43,1
71115,6ECNFdEh9FQC8Jlnf4cmT0,Like a Nightmare (Acoustic Instrumental),"Rebuild, Vol. 1",https://p.scdn.co/mp3-preview/23c8bd5a60fff4ef...,Deadset Society,future rock,220360,0.698,0.331,6,...,-0.027331,0.060340,-0.052983,-0.017829,-0.087992,0.148089,0.018954,-0.020380,43,11
71117,4CviUn1yqDAcebuVAafg7W,5ever,5ever / The Stars Up Above,https://p.scdn.co/mp3-preview/55209acc21047897...,Ssaliva,deconstructed club,125921,0.236,0.846,2,...,0.071296,-0.023229,-0.017327,-0.137258,0.097314,0.164446,-0.034294,-0.112003,43,0
71141,7j7PJ39u7AYV5lXdVG3Kko,Explode - Dope Stars INC Remix,The Perfect Cult,https://p.scdn.co/mp3-preview/33b848af89c6799e...,Deathstars,cyber metal,331013,0.522,0.944,1,...,0.058544,0.047814,-0.153424,0.020373,0.022648,0.121276,-0.012885,-0.148954,43,1


In [ ]:
#df_clust44_big.insert(34, "clusterLabelsSecond", predictions, True)

In [ ]:
#df_clust44_big

concat all the files

In [ ]:
Final_database = pd.concat([df_clust0_big, df_clust1_big,df_clust2_big,df_clust3_big,df_clust4_big,df_clust5_big,df_clust6_big,df_clust7_big,df_clust8_big,df_clust9_big,df_clust10_big,df_clust11_big,df_clust12_big,df_clust13_big,df_clust14_big,df_clust15_big,df_clust16_big,df_clust17_big,df_clust18_big,df_clust19_big,df_clust20_big,df_clust21_big,df_clust22_big,df_clust23_big,df_clust24_big,df_clust25_big,df_clust26_big,df_clust27_big,df_clust28_big,df_clust29_big,df_clust30_big,df_clust31_big,df_clust32_big,df_clust33_big,df_clust34_big,df_clust35_big,df_clust36_big,df_clust37_big,df_clust38_big,df_clust39_big,df_clust40_big,df_clust41_big,df_clust42_big,df_clust42_big,df_clust43_big]) #this is df_better_data with encoded values 
Final_database#concatonates all of the clusters into a large csv with all data to be putputted

,TrackID,TrackName,Album,PreviewURL,ArtistName,Genre,Duration,Danceability,Energy,Key,...,i,j,k,l,m,n,o,p,clusterLabels,clusterLabelsSecond
195,4JaidSe7p6grGM3VbeESI5,Divertimento No. 5 K. 439b: V. Polonaise,Mozart: Complete Divertimenti for Wind Ensemble,https://p.scdn.co/mp3-preview/b3e286a152779391...,Wolfgang Amadeus Mozart,classical,73413,0.407,0.1080,3,...,-0.017110,0.021865,0.006667,-0.006965,0.022210,0.192092,0.017031,-0.056889,0,4
204,0yt3XJZ85Wmv0cMSG9uvqM,﻿sonate Nr. 13 in B-dur Kv 333: Andante cantabile,Mozart the Double-Faced,https://p.scdn.co/mp3-preview/7dfe8eb3c465557b...,Wolfgang Amadeus Mozart,classical,645000,0.446,0.0234,3,...,0.001615,-0.000150,0.010567,-0.009108,-0.001090,0.164956,-0.007099,-0.097548,0,4
400,109BRkhzYBZr8OvIORR3Ue,elan vital,elan vital,https://p.scdn.co/mp3-preview/c2e4355922c80d64...,Daisuke Miyatani,j-ambient,218997,0.768,0.3240,6,...,-0.017424,-0.008118,0.015513,0.020855,-0.057284,0.137942,-0.053463,-0.064995,0,4
408,576bulYqFGM5hTMwXYHc3I,"Daisuki. - From ""Hensuki: Kawaikereba Hentai d...","Daisuki. (From ""Hensuki: Kawaikereba Hentai de...",https://p.scdn.co/mp3-preview/85c199b382007d27...,Mugi Piano,anime piano,96672,0.562,0.3390,8,...,0.035082,-0.018620,0.027790,0.003398,0.008963,0.175160,0.084573,-0.161992,0,4
468,21DG2ldRWCROUcufArlgeW,Living Walls,The 13th Hour,https://p.scdn.co/mp3-preview/9aa51afc1b5228ab...,Midnight Syndicate,ethereal wave,208093,0.146,0.1770,5,...,-0.006397,-0.009674,-0.026568,0.032321,-0.004519,0.191921,-0.013723,-0.060258,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71097,704d2awSVVDpY2b8Xn0N8e,Explode - Remix by Dope Stars Inc.,The Perfect Cult,https://p.scdn.co/mp3-preview/33b848af89c6799e...,Deathstars,cyber metal,331013,0.522,0.9440,1,...,0.058544,0.047814,-0.153424,0.020373,0.022648,0.121276,-0.012885,-0.148953,43,1
71115,6ECNFdEh9FQC8Jlnf4cmT0,Like a Nightmare (Acoustic Instrumental),"Rebuild, Vol. 1",https://p.scdn.co/mp3-preview/23c8bd5a60fff4ef...,Deadset Society,future rock,220360,0.698,0.3310,6,...,-0.027331,0.060340,-0.052983,-0.017829,-0.087992,0.148089,0.018954,-0.020380,43,11
71117,4CviUn1yqDAcebuVAafg7W,5ever,5ever / The Stars Up Above,https://p.scdn.co/mp3-preview/55209acc21047897...,Ssaliva,deconstructed club,125921,0.236,0.8460,2,...,0.071296,-0.023229,-0.017327,-0.137258,0.097314,0.164446,-0.034294,-0.112003,43,0
71141,7j7PJ39u7AYV5lXdVG3Kko,Explode - Dope Stars INC Remix,The Perfect Cult,https://p.scdn.co/mp3-preview/33b848af89c6799e...,Deathstars,cyber metal,331013,0.522,0.9440,1,...,0.058544,0.047814,-0.153424,0.020373,0.022648,0.121276,-0.012885,-0.148954,43,1


In [ ]:
from google.colab import files


In [ ]:
Final_database.to_csv('Final_database.csv', encoding = 'utf-8-sig') 
files.download('Final_database.csv') #download data to be used in next filter in the architecture 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Concatonate the clusters for easy addressing on the ones that have been double clustered 

##do this distance fucntion on a groupby call from the backend at runtime